# 테크닉 41 : 데이터를 읽어 들이고 이용 데이터를 수정하자

In [45]:
import pandas as pd

customer      = pd.read_csv("./dump_files/customer_join_with_membership_period.csv")
uselog_months = pd.read_csv("./dump_files/use_log_months.csv")

print("* customer :")
display(customer)

uselog_months = uselog_months.set_index(["customer_id", "연월"])
print("* uselog_months :")
display(uselog_months)

* customer :


,customer_id,name,class,class_name,price,gender,start_date,end_date,membership_period,is_deleted,campaign_id,campaign_name,mean,median,max,min,routine_flg
0,OA832399,XXXX,C01,0_종일,10500,F,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,4.833333,5.0,8,2,1
1,PL270116,XXXXX,C01,0_종일,10500,M,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,5.083333,5.0,7,3,1
2,OA974876,XXXXX,C01,0_종일,10500,M,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,4.583333,5.0,6,3,1
3,HD024127,XXXXX,C01,0_종일,10500,F,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,4.833333,4.5,7,2,1
4,HD661448,XXXXX,C03,2_야간,6000,F,2015-05-01,2019-04-30,48.7,0,CA1,2_일반,3.916667,4.0,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,0_종일,10500,M,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,8.000000,8.0,8,8,0
4188,HD246549,XXXXX,C01,0_종일,10500,F,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,10.000000,10.0,10,10,0
4189,GD037007,XXXXX,C03,2_야간,6000,M,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,8.000000,8.0,8,8,0
4190,OA953150,XXXXX,C01,0_종일,10500,M,2019-03-14,2019-04-30,1.6,0,CA1,2_일반,11.000000,11.0,11,11,0


* uselog_months :


,,count
customer_id,연월,
AS002855,201804,4
AS009013,201804,2
AS009373,201804,3
AS015315,201804,6
AS015739,201804,7
...,...,...
TS995853,201903,8
TS998593,201903,8
TS999079,201903,3


In [46]:
# 기준 연월과 해당 기준 연월의 1개월 전 이용 횟수를 집계한 데이터 작성
# '연월'값에 '201804'이전 데이터는 존재하지 않기 때문에, 연월값이 '201804'인 레코드를 조회에서 제외함

from pandasql import sqldf

uselog_base_month_cnt_and_bfr_month_cnt = sqldf("""
                                            SELECT Z.*
                                            FROM (
                                                SELECT  X.연월,
                                                        X.customer_id,
                                                        X.count AS count_0,
                                                        (SELECT Y1.count 
                                                         FROM   uselog_months Y1 
                                                         WHERE  Y1.customer_id = X.customer_id 
                                                                AND Y1.연월 = STRFTIME('%Y%m', DATETIME(SUBSTR(X.연월, 1, 4) || '-' || SUBSTR(X.연월, 5, 2) || '-' || '01', '-1 months'))
                                                        ) AS count_1        
                                                FROM   uselog_months X  
                                            ) Z
                                            WHERE Z.연월 >= '201805'     
                                        """)
print("* uselog_base_month_cnt_and_bfr_month_cnt :")
display(uselog_base_month_cnt_and_bfr_month_cnt)      

# # 기준 연월과 해당 기준 연월의 1개월 전 이용 횟수를 집계한 데이터 작성
# year_months = list(uselog_months["연월"].unique())
# print("year_months :", year_months)
# uselog = pd.DataFrame()

# for i in range(1, len(year_months)): # 1부터 반복을 도는 이유는 'year_months' 내 '201804'이전 데이터는 존재하지 않기 때문
    
#     tmp = uselog_months.loc[uselog_months["연월"] == year_months[i]]
#     tmp.rename(columns = {"count" : "count_0"}, inplace = True)
#     tmp_before = uselog_months.loc[uselog_months["연월"] == year_months[i-1]]
#     del tmp_before["연월"]
#     tmp_before.rename(columns = {"count" : "count_1"}, inplace = True)
#     tmp = pd.merge(tmp, tmp_before, on = "customer_id", how="left")
#     uselog = pd.concat([uselog, tmp], ignore_index=True)

# display(uselog.head())

* uselog_base_month_cnt_and_bfr_month_cnt :


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


# 테크닉 42 : 탈퇴 전월의 탈퇴 고객 데이터를 작성하자

In [47]:
# exit_customer, exit_uselog 데이터 프레임 생성
# exit_date : 회원 탈퇴 '신청 일자'

exit_customer = sqldf("""
                        SELECT customer_id,
                               name,
                               gender,
                               STRFTIME('%Y%m%d', start_date) AS start_date,
                               STRFTIME('%Y%m%d', end_date) AS end_date,
                               STRFTIME('%Y%m%d', DATETIME(DATETIME(end_date, 'start of month'), '-1 days')) AS exit_date,
                               STRFTIME('%Y%m', DATETIME(DATETIME(end_date, 'start of month'), '-1 days')) AS exit_date_YYYY_MM,
                               membership_period,
                               is_deleted,
                               class,
                               class_name,
                               price,
                               campaign_id,
                               campaign_name,
                               mean,
                               median,
                               max,
                               min,
                               routine_flg
                        FROM   customer
                        WHERE  is_deleted = 1
                     """)
print("* exit_customer :")
display(exit_customer)

exit_uselog = sqldf("""
                     SELECT    X.customer_id,
                               X.name,
                               X.is_deleted,
                               X.start_date,
                               X.end_date,
                               CAST(Y.연월 as text) AS base_month,
                               Y.count_0,
                               Y.count_1,
                               X.gender,
                               X.class,
                               X.class_name,
                               X.price,
                               X.campaign_id,
                               X.campaign_name,
                               X.mean,
                               X.median,
                               X.max,
                               X.min,
                               X.routine_flg,
                               X.membership_period
                     FROM      exit_customer X
                     LEFT JOIN uselog_base_month_cnt_and_bfr_month_cnt Y
                     ON        (X.exit_date_YYYY_MM = Y.연월
                                AND X.customer_id = Y.customer_id)
                   """)
print("* exit_uselog :")
exit_uselog = exit_uselog.dropna() # 결측치 제거
display(exit_uselog)

# # 탈퇴한 회원들의 탈퇴 전월 데이터 작성
# from dateutil.relativedelta import relativedelta

# exit_customer = customer.loc[customer["is_deleted"] == 1]
# exit_customer["exit_date"] = None
# exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
# # exit_customer.head()
# for i in range(len(exit_customer)):
    
#     exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
    
# #'exit_customer'데이터 셋 내 'exit_date'를 'YYYYMM'포맷의 문자열로 변경한 후, 이것을 '연월'이라는 컬럼으로 생성함
# exit_customer["연월"] = exit_customer["exit_date"].dt.strftime("%Y%m")
# uselog["연월"] = uselog["연월"].astype(str)
# exit_uselog = pd.merge(uselog, exit_customer, on = ["customer_id", "연월"], how="left")
# exit_uselog.head()

* exit_customer :


,customer_id,name,gender,start_date,end_date,exit_date,exit_date_YYYY_MM,membership_period,is_deleted,class,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg
0,TS511179,XXXXXX,F,20160501,20180430,20180331,201803,24.3,1,C01,0_종일,10500,CA1,2_일반,3.0,3.0,3,3,0
1,TS443736,XXXX,M,20160501,20180430,20180331,201803,24.3,1,C02,1_주간,7500,CA1,2_일반,3.0,3.0,3,3,0
2,HD542886,XX,M,20160501,20180430,20180331,201803,24.3,1,C01,0_종일,10500,CA1,2_일반,1.0,1.0,1,1,0
3,HD597545,XXXXX,F,20160601,20180531,20180430,201804,24.3,1,C03,2_야간,6000,CA1,2_일반,3.5,3.5,4,3,1
4,HI749296,XXXXX,M,20160601,20180531,20180430,201804,24.3,1,C01,0_종일,10500,CA1,2_일반,3.0,3.0,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,OA143589,XXXXX,F,20190115,20190228,20190131,201901,1.5,1,C03,2_야간,6000,CA1,2_일반,4.5,4.5,6,3,0
1346,GD367382,XXXX,M,20190115,20190331,20190228,201902,2.5,1,C01,0_종일,10500,CA1,2_일반,7.0,6.0,12,3,0
1347,OA034442,XXXXX,M,20190205,20190331,20190228,201902,1.8,1,C02,1_주간,7500,CA1,2_일반,7.0,7.0,11,3,0
1348,PL508278,XXXXX,M,20190209,20190331,20190228,201902,1.7,1,C03,2_야간,6000,CA1,2_일반,7.0,7.0,10,4,0


* exit_uselog :


,customer_id,name,is_deleted,start_date,end_date,base_month,count_0,count_1,gender,class,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg,membership_period
16,HD262908,XXXXX,1,20160801,20180630,201805,4.0,2.0,F,C03,2_야간,6000,CA1,2_일반,2.333333,2.0,4,1,0,23.3
22,HD944583,XXXX,1,20160901,20180731,201806,3.0,4.0,F,C02,1_주간,7500,CA1,2_일반,4.250000,4.0,6,3,1,23.3
24,AS047907,XXX,1,20161001,20180831,201807,4.0,5.0,M,C01,0_종일,10500,CA1,2_일반,4.000000,4.0,5,3,1,23.3
25,AS280744,XXX,1,20161001,20180930,201808,4.0,4.0,M,C02,1_주간,7500,CA1,2_일반,3.500000,4.0,4,1,1,24.3
27,OA308827,XXX,1,20161001,20180831,201807,3.0,3.0,F,C01,0_종일,10500,CA1,2_일반,2.200000,2.0,3,1,0,23.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,AS194818,XXXXX,1,20190105,20190331,201902,1.0,8.0,F,C02,1_주간,7500,CA1,2_일반,4.333333,4.0,8,1,1,2.8
1340,PL739722,XXXXXX,1,20190107,20190331,201902,6.0,7.0,M,C03,2_야간,6000,CA1,2_일반,5.000000,6.0,7,2,0,2.8
1341,HI815924,XXXXX,1,20190109,20190331,201902,5.0,9.0,F,C02,1_주간,7500,CA1,2_일반,5.333333,5.0,9,2,0,2.7
1343,AS484213,XXXXX,1,20190112,20190331,201902,4.0,10.0,F,C02,1_주간,7500,CA1,2_일반,5.000000,4.0,10,1,0,2.6


# 테크닉 43 : 지속 회원의 데이터를 작성하자

In [48]:
continue_customer = sqldf("""
                            SELECT  customer_id,
                                    name,
                                    gender,
                                    STRFTIME('%Y%m%d', start_date) AS start_date,
                                    STRFTIME('%Y%m%d', end_date) AS end_date,
                                    membership_period,
                                    is_deleted,
                                    class,
                                    class_name,
                                    price,
                                    campaign_id,
                                    campaign_name,
                                    mean,
                                    median,
                                    max,
                                    min,
                                    routine_flg
                            FROM   customer
                            WHERE  is_deleted = 0
                         """)
print("* continue_customer :")
display(continue_customer)

continue_uselog = sqldf("""
                     SELECT    X.customer_id,
                               X.name,
                               X.is_deleted,
                               X.start_date,
                               X.end_date,
                               CAST(Y.연월 as text) AS base_month,
                               Y.count_0,
                               Y.count_1,
                               X.gender,
                               X.class,
                               X.class_name,
                               X.price,
                               X.campaign_id,
                               X.campaign_name,
                               X.mean,
                               X.median,
                               X.max,
                               X.min,
                               X.routine_flg,
                               X.membership_period
                     FROM      continue_customer X
                     LEFT JOIN uselog_base_month_cnt_and_bfr_month_cnt Y
                     ON        X.customer_id = Y.customer_id
                   """)
print("* continue_uselog :")
display(continue_uselog)

* continue_customer :


,customer_id,name,gender,start_date,end_date,membership_period,is_deleted,class,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg
0,OA832399,XXXX,F,20150501,20190430,48.7,0,C01,0_종일,10500,CA1,2_일반,4.833333,5.0,8,2,1
1,PL270116,XXXXX,M,20150501,20190430,48.7,0,C01,0_종일,10500,CA1,2_일반,5.083333,5.0,7,3,1
2,OA974876,XXXXX,M,20150501,20190430,48.7,0,C01,0_종일,10500,CA1,2_일반,4.583333,5.0,6,3,1
3,HD024127,XXXXX,F,20150501,20190430,48.7,0,C01,0_종일,10500,CA1,2_일반,4.833333,4.5,7,2,1
4,HD661448,XXXXX,F,20150501,20190430,48.7,0,C03,2_야간,6000,CA1,2_일반,3.916667,4.0,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,HD676663,XXXX,M,20190314,20190430,1.6,0,C01,0_종일,10500,CA1,2_일반,8.000000,8.0,8,8,0
2838,HD246549,XXXXX,F,20190314,20190430,1.6,0,C01,0_종일,10500,CA1,2_일반,10.000000,10.0,10,10,0
2839,GD037007,XXXXX,M,20190314,20190430,1.6,0,C03,2_야간,6000,CA1,2_일반,8.000000,8.0,8,8,0
2840,OA953150,XXXXX,M,20190314,20190430,1.6,0,C01,0_종일,10500,CA1,2_일반,11.000000,11.0,11,11,0


* continue_uselog :


,customer_id,name,is_deleted,start_date,end_date,base_month,count_0,count_1,gender,class,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg,membership_period
0,OA832399,XXXX,0,20150501,20190430,201805,8,2.0,F,C01,0_종일,10500,CA1,2_일반,4.833333,5.0,8,2,1,48.7
1,OA832399,XXXX,0,20150501,20190430,201806,5,8.0,F,C01,0_종일,10500,CA1,2_일반,4.833333,5.0,8,2,1,48.7
2,OA832399,XXXX,0,20150501,20190430,201807,4,5.0,F,C01,0_종일,10500,CA1,2_일반,4.833333,5.0,8,2,1,48.7
3,OA832399,XXXX,0,20150501,20190430,201808,5,4.0,F,C01,0_종일,10500,CA1,2_일반,4.833333,5.0,8,2,1,48.7
4,OA832399,XXXX,0,20150501,20190430,201809,3,5.0,F,C01,0_종일,10500,CA1,2_일반,4.833333,5.0,8,2,1,48.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27417,HD676663,XXXX,0,20190314,20190430,201903,8,NaN,M,C01,0_종일,10500,CA1,2_일반,8.000000,8.0,8,8,0,1.6
27418,HD246549,XXXXX,0,20190314,20190430,201903,10,NaN,F,C01,0_종일,10500,CA1,2_일반,10.000000,10.0,10,10,0,1.6
27419,GD037007,XXXXX,0,20190314,20190430,201903,8,NaN,M,C03,2_야간,6000,CA1,2_일반,8.000000,8.0,8,8,0,1.6
27420,OA953150,XXXXX,0,20190314,20190430,201903,11,NaN,M,C01,0_종일,10500,CA1,2_일반,11.000000,11.0,11,11,0,1.6


In [49]:
# 'continue_uselog'에 언더샘플링 실시

continue_uselog = continue_uselog.sample(frac = 1).reset_index(drop = True)
continue_uselog = continue_uselog.drop_duplicates(subset = 'customer_id')
print("* continue_uselog :")
display(continue_uselog)

* continue_uselog :


,customer_id,name,is_deleted,start_date,end_date,base_month,count_0,count_1,gender,class,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg,membership_period
0,TS143759,XXX,0,20190104,20190430,201903,8,6.0,F,C01,0_종일,10500,CA1,2_일반,7.666667,8.0,9,6,1,3.9
1,PL863297,XXXXX,0,20190105,20190430,201903,7,7.0,M,C01,0_종일,10500,CA1,2_일반,6.666667,7.0,7,6,1,3.8
2,HD775663,XXXXX,0,20150901,20190430,201901,5,6.0,M,C03,2_야간,6000,CA1,2_일반,4.916667,5.0,7,3,1,44.6
3,AS499352,XXXXX,0,20170101,20190430,201805,4,7.0,F,C02,1_주간,7500,CA1,2_일반,5.666667,5.0,9,4,1,28.3
4,HD961587,XX,0,20160701,20190430,201806,7,4.0,F,C01,0_종일,10500,CA2,0_입회비반액할인,4.500000,4.0,7,3,1,34.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24427,PL644166,XXX,0,20190213,20190430,201903,9,5.0,M,C01,0_종일,10500,CA1,2_일반,7.000000,7.0,9,5,1,2.5
24613,AS433133,XXXX,0,20190208,20190430,201902,11,NaN,M,C03,2_야간,6000,CA1,2_일반,9.500000,9.5,11,8,1,2.7
25495,IK921404,XXXX,0,20190201,20190430,201902,10,NaN,F,C02,1_주간,7500,CA1,2_일반,9.000000,9.0,10,8,1,2.9
25946,IK626454,XXXXX,0,20190307,20190430,201903,9,NaN,F,C03,2_야간,6000,CA1,2_일반,9.000000,9.0,9,9,0,1.8


In [50]:
# 'continue_uselog'와 'exit_uselog'를 union all 실시

predict_data = pd.concat([continue_uselog, exit_uselog], ignore_index = True)
print("* predict_data :")
display(predict_data)

* predict_data :


,customer_id,name,is_deleted,start_date,end_date,base_month,count_0,count_1,gender,class,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg,membership_period
0,TS143759,XXX,0,20190104,20190430,201903,8.0,6.0,F,C01,0_종일,10500,CA1,2_일반,7.666667,8.0,9,6,1,3.9
1,PL863297,XXXXX,0,20190105,20190430,201903,7.0,7.0,M,C01,0_종일,10500,CA1,2_일반,6.666667,7.0,7,6,1,3.8
2,HD775663,XXXXX,0,20150901,20190430,201901,5.0,6.0,M,C03,2_야간,6000,CA1,2_일반,4.916667,5.0,7,3,1,44.6
3,AS499352,XXXXX,0,20170101,20190430,201805,4.0,7.0,F,C02,1_주간,7500,CA1,2_일반,5.666667,5.0,9,4,1,28.3
4,HD961587,XX,0,20160701,20190430,201806,7.0,4.0,F,C01,0_종일,10500,CA2,0_입회비반액할인,4.500000,4.0,7,3,1,34.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3889,AS194818,XXXXX,1,20190105,20190331,201902,1.0,8.0,F,C02,1_주간,7500,CA1,2_일반,4.333333,4.0,8,1,1,2.8
3890,PL739722,XXXXXX,1,20190107,20190331,201902,6.0,7.0,M,C03,2_야간,6000,CA1,2_일반,5.000000,6.0,7,2,0,2.8
3891,HI815924,XXXXX,1,20190109,20190331,201902,5.0,9.0,F,C02,1_주간,7500,CA1,2_일반,5.333333,5.0,9,2,0,2.7
3892,AS484213,XXXXX,1,20190112,20190331,201902,4.0,10.0,F,C02,1_주간,7500,CA1,2_일반,5.000000,4.0,10,1,0,2.6


# 테크닉 44 : 예측할 달의 재적 기간을 작성하자

In [51]:
# 'predict_data'에 'period'컬럼 추가

predict_data = sqldf("""
                        SELECT customer_id,
                               name,
                               is_deleted,
                               start_date,
                               end_date,
                               base_month,
                               CAST((julianday(SUBSTR(base_month, 1, 4) || '-' || SUBSTR(base_month, 5, 2) || '-' || '01') 
                               - julianday(SUBSTR(start_date, 1, 4) || '-' || SUBSTR(start_date, 5, 2) || '-' || SUBSTR(start_date, 7, 2))) / 30 AS INTEGER) AS period,
                               count_0,
                               count_1,
                               gender,
                               class,
                               class_name,
                               price,
                               campaign_id,
                               campaign_name,
                               mean,
                               median,
                               max,
                               min,
                               routine_flg,
                               membership_period
                        FROM   predict_data
                    """)
print("* predict_data :")
display(predict_data)

* predict_data :


,customer_id,name,is_deleted,start_date,end_date,base_month,period,count_0,count_1,gender,...,class_name,price,campaign_id,campaign_name,mean,median,max,min,routine_flg,membership_period
0,TS143759,XXX,0,20190104,20190430,201903,1,8.0,6.0,F,...,0_종일,10500,CA1,2_일반,7.666667,8.0,9,6,1,3.9
1,PL863297,XXXXX,0,20190105,20190430,201903,1,7.0,7.0,M,...,0_종일,10500,CA1,2_일반,6.666667,7.0,7,6,1,3.8
2,HD775663,XXXXX,0,20150901,20190430,201901,40,5.0,6.0,M,...,2_야간,6000,CA1,2_일반,4.916667,5.0,7,3,1,44.6
3,AS499352,XXXXX,0,20170101,20190430,201805,16,4.0,7.0,F,...,1_주간,7500,CA1,2_일반,5.666667,5.0,9,4,1,28.3
4,HD961587,XX,0,20160701,20190430,201806,23,7.0,4.0,F,...,0_종일,10500,CA2,0_입회비반액할인,4.500000,4.0,7,3,1,34.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3889,AS194818,XXXXX,1,20190105,20190331,201902,0,1.0,8.0,F,...,1_주간,7500,CA1,2_일반,4.333333,4.0,8,1,1,2.8
3890,PL739722,XXXXXX,1,20190107,20190331,201902,0,6.0,7.0,M,...,2_야간,6000,CA1,2_일반,5.000000,6.0,7,2,0,2.8
3891,HI815924,XXXXX,1,20190109,20190331,201902,0,5.0,9.0,F,...,1_주간,7500,CA1,2_일반,5.333333,5.0,9,2,0,2.7
3892,AS484213,XXXXX,1,20190112,20190331,201902,0,4.0,10.0,F,...,1_주간,7500,CA1,2_일반,5.000000,4.0,10,1,0,2.6


# 테크닉 45 : 결측치를 제거하자

In [52]:
# 'predict_data'에 대한 결측치를 파악한 뒤, 결측치 제거 실시

print("* null of predict_data :")
display(predict_data.isna().sum())

predict_data = predict_data.dropna()

print("* count by is_deleted :")
display(sqldf("""
                SELECT is_deleted,
                       COUNT(1) AS cnt
                FROM   predict_data
                GROUP BY is_deleted
             """)
       )

* null of predict_data :


customer_id            0
name                   0
is_deleted             0
start_date             0
end_date               0
base_month             0
period                 0
count_0                0
count_1              202
gender                 0
class                  0
class_name             0
price                  0
campaign_id            0
campaign_name          0
mean                   0
median                 0
max                    0
min                    0
routine_flg            0
membership_period      0
dtype: int64

* count by is_deleted :


,is_deleted,cnt
0,0,2640
1,1,1052


# 테크닉 46 : 문자열 변수를 처리할 수 있게 가공하자

In [53]:
# 카테고리 변수(class_name, campaign_name, gender)에 대한 더미 변수 생성 실시

target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"] # 독립변수 : "campaign_name", "class_name", "gender", "count_1", "routine_flg", "period"
                                                                                                         # 종속변수 : "is_deleted"
predict_data = predict_data[target_col]
print("* predict_data :")
display(predict_data)

predict_data_with_dummy_cols = pd.get_dummies(predict_data)
del predict_data_with_dummy_cols["campaign_name_2_일반"]
del predict_data_with_dummy_cols["class_name_2_야간"]
del predict_data_with_dummy_cols["gender_M"]
print("* predict_data_with_dummy_cols :")
display(predict_data_with_dummy_cols)

* predict_data :


,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,2_일반,0_종일,F,6.0,1,1,0
1,2_일반,0_종일,M,7.0,1,1,0
2,2_일반,2_야간,M,6.0,1,40,0
3,2_일반,1_주간,F,7.0,1,16,0
4,0_입회비반액할인,0_종일,F,4.0,1,23,0
...,...,...,...,...,...,...,...
3889,2_일반,1_주간,F,8.0,1,0,1
3890,2_일반,2_야간,M,7.0,0,0,1
3891,2_일반,1_주간,F,9.0,0,0,1
3892,2_일반,1_주간,F,10.0,0,0,1


* predict_data_with_dummy_cols :


,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,6.0,1,1,0,0,0,1,0,1
1,7.0,1,1,0,0,0,1,0,0
2,6.0,1,40,0,0,0,0,0,0
3,7.0,1,16,0,0,0,0,1,1
4,4.0,1,23,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...
3889,8.0,1,0,1,0,0,0,1,1
3890,7.0,0,0,1,0,0,0,0,0
3891,9.0,0,0,1,0,0,0,1,1
3892,10.0,0,0,1,0,0,0,1,1


# 테크닉 47 : 의사결정 트리를 사용해서 탈퇴 예측 모델을 구축하자

In [57]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit_customer = predict_data_with_dummy_cols.loc[predict_data_with_dummy_cols["is_deleted"] == 1]
continue_customer = predict_data_with_dummy_cols.loc[predict_data_with_dummy_cols["is_deleted"] == 0].sample(n = len(exit_customer)) # 유지 고객 데이터 개수와 탈퇴 고객 데이터 개수를 동일하게 맞춤
print("* length of exit_customer :", len(exit_customer))
print("* length of continue_customer :", len(continue_customer), "\n")

X = pd.concat([exit_customer, continue_customer], ignore_index = True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state = 0)
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print("* y_test_pred :", "\n")
print(y_test_pred)

* length of exit_customer : 1052
* length of continue_customer : 1052 

* y_test_pred : 

[0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0
 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 1
 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 0
 1 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0
 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0
 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 1
 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1
 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1
 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 0
 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 1
 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1
 0 0 1 0 1

In [58]:
results_about_test = pd.DataFrame({"y_test" : y_test, "y_pred" : y_test_pred})
print("* results_about_test :")
display(results_about_test)

* results_about_test :


,y_test,y_pred
1906,0,0
1487,0,0
2094,0,0
2057,0,0
395,1,1
...,...,...
713,1,1
2100,0,0
1005,1,1
2046,0,0


# 테크닉 48 : 예측 모델을 평가하고 모델을 튜닝해 보자